<a href="https://colab.research.google.com/github/vinee-sha/Social-Distancing-Tracker/blob/main/TrainHuman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
!unzip "/content/drive/MyDrive/HumanDataset.zip" -d "/content"

Archive:  /content/drive/MyDrive/HumanDataset.zip
   creating: /content/HumanDataset/Train/
   creating: /content/HumanDataset/Train/Human/
  inflating: /content/HumanDataset/Train/Human/brushing_teeth_078.jpg  
  inflating: /content/HumanDataset/Train/Human/brushing_teeth_079.jpg  
  inflating: /content/HumanDataset/Train/Human/brushing_teeth_080.jpg  
  inflating: /content/HumanDataset/Train/Human/brushing_teeth_081.jpg  
  inflating: /content/HumanDataset/Train/Human/brushing_teeth_082.jpg  
  inflating: /content/HumanDataset/Train/Human/brushing_teeth_083.jpg  
  inflating: /content/HumanDataset/Train/Human/brushing_teeth_085.jpg  
  inflating: /content/HumanDataset/Train/Human/brushing_teeth_086.jpg  
  inflating: /content/HumanDataset/Train/Human/brushing_teeth_087.jpg  
  inflating: /content/HumanDataset/Train/Human/brushing_teeth_088.jpg  
  inflating: /content/HumanDataset/Train/Human/brushing_teeth_175.jpg  
  inflating: /content/HumanDataset/Train/Human/brushing_teeth_176.jp

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
train_dir = "/content/HumanDataset/Train"
validation_dir = "/content/HumanDataset/Validation"

In [12]:
train_humans_dir = os.path.join(train_dir, 'Human')
validation_humans_dir = os.path.join(validation_dir, 'Human')

In [13]:
total_train = len(os.listdir(train_humans_dir)) 
total_val = len(os.listdir(validation_humans_dir)) 
print(total_train, total_val)

925 581


In [18]:
# Model Parameters
BATCH_SIZE = 120
IMG_SHAPE = 150
EPOCHS = 100

MODEL CREATION

In [16]:
# Initialize the CNN
classifier = Sequential()

In [19]:
# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape=(IMG_SHAPE, IMG_SHAPE, 3), activation='relu'))

In [20]:
# Step 2 - Pooling
classifier.add(MaxPooling2D(2, 2))

In [21]:
# Step 1 & 2 with more hidden layers
classifier.add(Conv2D(64, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(2, 2))

classifier.add(Conv2D(128, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(2, 2))

classifier.add(Conv2D(128, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(2, 2))

In [22]:
# Step 3 - Dropout
classifier.add(Dropout(0.5))

In [23]:
# Step 4 - Flattening
classifier.add(Flatten())

In [24]:
# Step 5 - Full connection
classifier.add(Dense(units=512, activation='relu'))

In [27]:
# Step 6 - Output layer
classifier.add(Dense(units=1, activation='sigmoid'))

In [28]:
# Compiling the CNN
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [29]:
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 128)      

Data Augmentation and Preparation

In [32]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2, 
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
) 

In [37]:
training_set = train_datagen.flow_from_directory(
    batch_size=BATCH_SIZE,
    directory=train_dir,
    shuffle=True,
    target_size=(IMG_SHAPE, IMG_SHAPE),
    class_mode='binary'
)

Found 925 images belonging to 1 classes.


In [38]:
validation_datagen = ImageDataGenerator(rescale=1./255)

In [39]:
validation_set = validation_datagen.flow_from_directory(
    batch_size=BATCH_SIZE,
    directory=validation_dir,
    target_size=(IMG_SHAPE, IMG_SHAPE),
    class_mode='binary'
)

Found 581 images belonging to 1 classes.


In [40]:
history = classifier.fit_generator(
    training_set,
    steps_per_epoch=int(np.ceil(total_train / float(BATCH_SIZE))),
    epochs=EPOCHS,
    validation_data=validation_set,
    validation_steps=int(np.ceil(total_val / float(BATCH_SIZE)))
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
8/8 [==============================] - 45s 1s/step - loss: 0.9521 - accuracy: 0.0000e+00 - val_loss: 0.9318 - val_accuracy: 0.0000e+00
Epoch 2/100
8/8 [==============================] - 11s 1s/step - loss: 0.9276 - accuracy: 0.0000e+00 - val_loss: 0.9226 - val_accuracy: 0.0000e+00
Epoch 3/100
8/8 [==============================] - 11s 1s/step - loss: 0.9188 - accuracy: 0.0000e+00 - val_loss: 0.9136 - val_accuracy: 0.0000e+00
Epoch 4/100
8/8 [==============================] - 11s 1s/step - loss: 0.9097 - accuracy: 0.0000e+00 - val_loss: 0.9047 - val_accuracy: 0.0000e+00
Epoch 5/100
8/8 [==============================] - 11s 1s/step - loss: 0.9008 - accuracy: 0.0000e+00 - val_loss: 0.8960 - val_accuracy: 0.0000e+00
Epoch 6/100
8/8 [==============================] - 11s 1s/step - loss: 0.8921 - accuracy: 0.0000e+00 - val_loss: 0.8874 - val_accuracy: 0.0000e+00
Epoch 7/100
8/8 [==============================] - 11s 1s/step - loss: 0.8838 - accuracy: 0.0000e+00 - val_loss: 0.878